## Introduction to Sentence Transformers and MLflow

Welcome to our tutorial on leveraging **Sentence Transformers** with **MLflow** for advanced natural language processing and model management.

### Learning Objectives

- Set up a pipeline for sentence embeddings with `sentence-transformers`.
- Log models and configurations using MLflow.
- Understand and apply model signatures in MLflow to `sentence-transformers`.
- Deploy and use models for inference with MLflow's features.

To learn more, expand the **details** sections throughout this tutorial.

<details>
    <div>
        <h4>What are Sentence Transformers?</h4>
        <p>Sentence Transformers, an extension of the Hugging Face Transformers library, are designed for generating semantically rich sentence embeddings. They utilize models like BERT and RoBERTa, fine-tuned for tasks such as semantic search and text clustering, producing high-quality sentence-level embeddings.</p>
    </div>
    <div>
        <h4>Benefits of Integrating MLflow with Sentence Transformers</h4>
        <p>Combining MLflow with Sentence Transformers enhances NLP projects by:</p>
        <ul>
            <li>Streamlining experiment management and logging.</li>
            <li>Offering better control over model versions and configurations.</li>
            <li>Ensuring reproducibility of results and model predictions.</li>
            <li>Simplifying the deployment process in production environments.</li>
        </ul>
        <p>This integration empowers efficient tracking, management, and deployment of NLP applications.</p>
    </div>
</details>
<br/>

In [1]:
# Disable tokenizers warnings when constructing pipelines
%env TOKENIZERS_PARALLELISM=false

import warnings

# Disable a few less-than-useful UserWarnings from setuptools and pydantic
warnings.filterwarnings("ignore", category=UserWarning)

env: TOKENIZERS_PARALLELISM=false


### Setting Up the Environment for Sentence Embedding

Begin your journey with Sentence Transformers and MLflow by establishing the core working environment.

#### Key Steps for Initialization

- Import necessary libraries: `SentenceTransformer` and `mlflow`.
- Initialize the `"all-MiniLM-L6-v2"` Sentence Transformer model.

<details>
    <div>
        <h4>Model Initialization</h4>
        <p>The compact and efficient <code>"all-MiniLM-L6-v2"</code> model is chosen for its effectiveness in generating meaningful sentence embeddings. Explore more models at the <a href="https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=trending">Hugging Face Hub</a>.</p>
    </div>
    <div>
        <h4>Purpose of the Model</h4>
        <p>This model excels in transforming sentences into semantically rich embeddings, applicable in various NLP tasks like semantic search and clustering.</p>
    </div>
</details>
<br/>

In [2]:
import mlflow
from sentence_transformers import SentenceTransformer


model = SentenceTransformer("all-MiniLM-L6-v2")

### Defining the Model Signature with MLflow
Defining the model signature is a crucial step in setting up our Sentence Transformer model for consistent and expected behavior during inference.

<details>
    <div>
        <h4>Steps for Signature Definition</h4>
        <ul>
            <li><strong>Prepare Example Sentences</strong>: Define example sentences to demonstrate the model's input and output formats.</li>
            <li><strong>Generate Model Signature</strong>: Use the <code>mlflow.models.infer_signature</code> function with the model's input and output to automatically define the signature.</li>
        </ul>
    </div>
    <div>
        <h4>Importance of the Model Signature</h4>
        <ul>
            <li><strong>Clarity in Data Formats</strong>: Ensures clear documentation of the data types and structures the model expects and produces.</li>
            <li><strong>Model Deployment and Usage</strong>: Crucial for deploying models to production, ensuring the model receives inputs in the correct format and produces expected outputs.</li>
            <li><strong>Error Prevention</strong>: Helps in preventing errors during model inference by enforcing consistent data formats.</li>
        </ul>
        <p><strong>NOTE</strong>: The <code>List[str]</code> input type is equivalent at inference time to <code>str</code>. The MLflow flavor uses a <code>ColSpec[str]</code> definition for the input type.</p>
    </div>
</details>
<br/>

In [3]:
example_sentences = ["A sentence to encode.", "Another sentence to encode."]

# Infer the signature of the custom model by providing an input example and the resultant prediction output.
# We're not including any custom inference parameters in this example, but you can include them as a third argument
# to infer_signature(), as you will see in the advanced tutorials for Sentence Transformers.
signature = mlflow.models.infer_signature(
    model_input=example_sentences,
    model_output=model.encode(example_sentences),
)

# Visualize the signature
signature

inputs: 
  [string]
outputs: 
  [Tensor('float32', (-1, 384))]
params: 
  None

### Creating an experiment

We create a new MLflow Experiment so that the run we're going to log our model to does not log to the default experiment and instead has its own contextually relevant entry.

In [4]:
# If you are running this tutorial in local mode, leave the next line commented out.
# Otherwise, uncomment the following line and set your tracking uri to your local or remote tracking server.

# mlflow.set_tracking_uri("http://127.0.0.1:8080")

mlflow.set_experiment("Introduction to Sentence Transformers")

<Experiment: artifact_location='file:///Users/benjamin.wilson/repos/mlflow-fork/mlflow/docs/source/llms/sentence-transformers/tutorials/quickstart/mlruns/469990615226680434', creation_time=1701280211449, experiment_id='469990615226680434', last_update_time=1701280211449, lifecycle_stage='active', name='Introduction to Sentence Transformers', tags={}>

### Logging the Sentence Transformer Model with MLflow

Logging the model in MLflow is essential for tracking, version control, and deployment, following the initialization and signature definition of our Sentence Transformer model.

<details>
    <div>
        <h4>Steps for Logging the Model</h4>
        <ul>
            <li><strong>Start an MLflow Run</strong>: Initiate a new run with <code>mlflow.start_run()</code>, grouping all logging operations.</li>
            <li><strong>Log the Model</strong>: Use <code>mlflow.sentence_transformers.log_model</code> to log the model, providing the model object, artifact path, signature, and an input example.</li>
        </ul>
    </div>
    <div>
        <h4>Importance of Model Logging</h4>
        <ul>
            <li><strong>Model Management</strong>: Facilitates the model's lifecycle management from training to deployment.</li>
            <li><strong>Reproducibility and Tracking</strong>: Enables tracking of model versions and ensures reproducibility.</li>
            <li><strong>Ease of Deployment</strong>: Simplifies deployment by allowing models to be easily deployed for inference.</li>
        </ul>
    </div>
</details>
<br/>

In [5]:
with mlflow.start_run():
    logged_model = mlflow.sentence_transformers.log_model(
        model=model,
        artifact_path="sbert_model",
        signature=signature,
        input_example=example_sentences,
    )

### Loading the Model and Testing Inference

After logging the Sentence Transformer model in MLflow, we demonstrate how to load and test it for real-time inference.

<details>
    <div>
        <h4>Loading the Model as a PyFunc</h4>
        <ul>
            <li><strong>Why PyFunc</strong>: Load the logged model using <code>mlflow.pyfunc.load_model</code> for seamless integration into Python-based services or applications.</li>
            <li><strong>Model URI</strong>: Use the <code>logged_model.model_uri</code> to accurately locate and load the model from MLflow.</li>
        </ul>
    </div>
    <div>
        <h4>Conducting Inference Tests</h4>
        <ul>
            <li><strong>Test Sentences</strong>: Define sentences to test the model's embedding generation capabilities.</li>
            <li><strong>Performing Predictions</strong>: Use the model's <code>predict</code> method with test sentences to obtain embeddings.</li>
            <li><strong>Printing Embedding Lengths</strong>: Verify embedding generation by checking the length of embedding arrays, corresponding to the dimensionality of each sentence representation.</li>
        </ul>
    </div>
    <div>
        <h4>Importance of Inference Testing</h4>
        <ul>
            <li><strong>Model Validation</strong>: Confirm the model's expected behavior and data processing capability upon loading.</li>
            <li><strong>Deployment Readiness</strong>: Validate the model's readiness for real-time integration into application services.</li>
        </ul>
    </div>
</details>
<br/>

In [6]:
inference_test = ["I enjoy pies of both apple and cherry.", "I prefer cookies."]

# Load our custom model by providing the uri for where the model was logged.
loaded_model_pyfunc = mlflow.pyfunc.load_model(logged_model.model_uri)

# Perform a quick test to ensure that our loaded model generates the correct output
embeddings_test = loaded_model_pyfunc.predict(inference_test)

# Verify that the output is a list of lists of floats (our expected output format)
print(f"The return structure length is: {len(embeddings_test)}")

for i, embedding in enumerate(embeddings_test):
    print(f"The size of embedding {i + 1} is: {len(embeddings_test[i])}")

The return structure length is: 2
The size of embedding 1 is: 384
The size of embedding 2 is: 384


### Displaying Samples of Generated Embeddings
Examine the content of embeddings to verify their quality and understand the model's output.

<details>
    <div>
        <h4>Inspecting the Embedding Samples</h4>
        <ul>
            <li><strong>Purpose of Sampling</strong>: Inspect a sample of the entries in each embedding to understand the vector representations generated by the model.</li>
            <li><strong>Printing Embedding Samples</strong>: Print the first 10 entries of each embedding vector using <code>embedding[:10]</code> to get a glimpse into the model's output.</li>
        </ul>
    </div>
    <div>
        <h4>Why Sampling is Important</h4>
        <ul>
            <li><strong>Quality Check</strong>: Sampling provides a quick way to verify the embeddings' quality and ensures they are meaningful and non-degenerate.</li>
            <li><strong>Understanding Model Output</strong>: Seeing parts of the embedding vectors offers an intuitive understanding of the model's output, beneficial for debugging and development.</li>
        </ul>
    </div>
</details>
<br/>

In [7]:
for i, embedding in enumerate(embeddings_test):
    print(f"The sample of the first 10 entries in embedding {i + 1} is: {embedding[:10]}")

The sample of the first 10 entries in embedding 1 is: [ 0.04866192 -0.03687946  0.02408808  0.03534171 -0.12739632  0.00999414
  0.07135344 -0.01433522  0.04296691 -0.00654414]
The sample of the first 10 entries in embedding 2 is: [-0.03879027 -0.02373698  0.01314073  0.03589077 -0.01641303 -0.0857707
  0.08282158 -0.03173266  0.04507608  0.02777079]


### Native Model Loading in MLflow for Extended Functionality
Explore the full range of Sentence Transformer functionalities with MLflow's support for native model loading.

<details>
    <div>
        <h4>Why Support Native Loading?</h4>
        <ul>
            <li><strong>Access to Native Functionalities</strong>: Native loading unlocks all the features of the Sentence Transformer model, essential for advanced NLP tasks.</li>
            <li><strong>Loading the Model Natively</strong>: Use <code>mlflow.sentence_transformers.load_model</code> to load the model with its full capabilities, enhancing flexibility and efficiency.</li>
        </ul>
    </div>
    <div>
        <h4>Generating Embeddings Using Native Model</h4>
        <ul>
            <li><strong>Model Encoding</strong>: Employ the model's native <code>encode</code> method to generate embeddings, taking advantage of optimized functionality.</li>
            <li><strong>Importance of Native Encoding</strong>: Native encoding ensures the utilization of the model's full embedding generation capabilities, suitable for large-scale or complex NLP applications.</li>
        </ul>
    </div>
</details>
<br/>

In [8]:
# Load the saved model as a native Sentence Transformers model (unlike above, where we loaded as a generic python function)
loaded_model_native = mlflow.sentence_transformers.load_model(logged_model.model_uri)

# Use the native model to generate embeddings by calling encode() (unlike for the generic python function which uses the single entrypoint of `predict`)
native_embeddings = loaded_model_native.encode(inference_test)

for i, embedding in enumerate(native_embeddings):
    print(
        f"The sample of the native library encoding call for embedding {i + 1} is: {embedding[:10]}"
    )

2023/11/30 15:50:24 INFO mlflow.sentence_transformers: 'runs:/eeab3c1b13594fdea13e07585b1c0596/sbert_model' resolved as 'file:///Users/benjamin.wilson/repos/mlflow-fork/mlflow/docs/source/llms/sentence-transformers/tutorials/quickstart/mlruns/469990615226680434/eeab3c1b13594fdea13e07585b1c0596/artifacts/sbert_model'


The sample of the native library encoding call for embedding 1 is: [ 0.04866192 -0.03687946  0.02408808  0.03534171 -0.12739632  0.00999414
  0.07135344 -0.01433522  0.04296691 -0.00654414]
The sample of the native library encoding call for embedding 2 is: [-0.03879027 -0.02373698  0.01314073  0.03589077 -0.01641303 -0.0857707
  0.08282158 -0.03173266  0.04507608  0.02777079]


### Conclusion: Embracing the Power of Sentence Transformers with MLflow

As we reach the end of our Introduction to Sentence Transformers tutorial, we have successfully navigated the basics of integrating the Sentence Transformers library with MLflow. This foundational knowledge sets the stage for more advanced and specialized applications in the field of Natural Language Processing (NLP).

#### Recap of Key Learnings

1. **Integration Basics**: We covered the essential steps of loading and logging a Sentence Transformer model using MLflow. This process demonstrated the simplicity and effectiveness of integrating cutting-edge NLP tools within MLflow's ecosystem.

2. **Signature and Inference**: Through the creation of a model signature and the execution of inference tasks, we showcased how to operationalize the Sentence Transformer model, ensuring that it's ready for real-world applications.

3. **Model Loading and Prediction**: We explored two ways of loading the model - as a PyFunc model and using the native Sentence Transformers loading mechanism. This dual approach highlighted the versatility of MLflow in accommodating different model interaction methods.

4. **Embeddings Exploration**: By generating and examining sentence embeddings, we glimpsed the transformative potential of transformer models in capturing semantic information from text.

#### Looking Ahead

- **Expanding Horizons**: While this tutorial focused on the foundational aspects of Sentence Transformers and MLflow, there's a whole world of advanced applications waiting to be explored. From semantic similarity analysis to paraphrase mining, the potential use cases are vast and varied.

- **Continued Learning**: We strongly encourage you to delve into the other tutorials in this series, which dive deeper into more intriguing use cases like similarity analysis, semantic search, and paraphrase mining. These tutorials will provide you with a broader understanding and more practical applications of Sentence Transformers in various NLP tasks.

#### Final Thoughts

The journey into NLP with Sentence Transformers and MLflow is just beginning. With the skills and insights gained from this tutorial, you are well-equipped to explore more complex and exciting applications. The integration of advanced NLP models with MLflow's robust management and deployment capabilities opens up new avenues for innovation and exploration in the field of language understanding and beyond.

Thank you for joining us on this introductory journey, and we look forward to seeing how you apply these tools and concepts in your NLP endeavors!